##### QnA chatcot using RAG. context is standford lectures. this is a take home assignment for an application(Ema)

In [5]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=getpass.getpass()

 ········


# Data Ingestion

In [4]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/introduction/",
                                "https://stanford-cs324.github.io/winter2022/lectures/legality/",
                                "https://stanford-cs324.github.io/winter2022/lectures/modeling/",
                                "https://stanford-cs324.github.io/winter2022/lectures/training/",),
                    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap")))) 

### we are using 4 lectures: intro, legality, modeling and training. 
### since all the content was inside the main class "main-content-wrap" we used that


In [10]:
document=loader.load() ## making documents containing all the information
len(document)

4

# Transform

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300, add_start_index=True)

In [28]:
split=splitter.split_documents(document) # splitting documents into chunks 

In [29]:
len(split)

125

In [30]:
split[0].page_content

'LecturesIntroduction \\[\\newcommand{\\sV}{\\mathcal{V}} \\newcommand{\\nl}[1]{\\textsf{#1}} \\newcommand{\\generate}[1]{\\stackrel{#1}{\\rightsquigarrow}}\\]Welcome to CS324! This is a new course on understanding and developing large language models.What is a language model?A brief historyWhy does this course exist?Structure of this course  What is a language model?The classic definition of a language model (LM) is a probability distribution over sequences of tokens. Suppose we have a vocabulary \\(\\sV\\) of a set of tokens. A language model \\(p\\) assigns each sequence of tokens \\(x_1, \\dots, x_L \\in \\sV\\) a probability (a number between 0 and 1):\\[p(x_1, \\dots, x_L).\\]The probability intuitively tells us how “good” a sequence of tokens is. For example, if the vocabulary is \\(\\sV = \\{ \\nl{ate}, \\nl{ball}, \\nl{cheese}, \\nl{mouse}, \\nl{the} \\}\\), the language model might assign (demo):\\[p(\\nl{the}, \\nl{mouse}, \\nl{ate}, \\nl{the}, \\nl{cheese}) = 0.02,\\] \\[p(

# Retriever

In [34]:
from langchain_core.prompts import ChatPromptTemplate

system_message=(""" You are a question-answer assistant.
                    Answer the questions based on the context provided.
                    Keep the answers as accurate and concise as possible.
                    If the answer is not present in the context then say its not available in the context.
                    \n\n
                    {context}.""")

prompt=ChatPromptTemplate.from_messages([("system",system_message),
                                        ("human","{input}")])


In [35]:
from langchain_community.llms import Ollama

model=Ollama(model="llama3")

In [37]:
from langchain.chains.combine_documents import create_stuff_documents_chain

stuff_doc_chain=create_stuff_documents_chain(model,prompt) #creating stuff document chain or basically chaining llm and prompt

In [39]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

vector_storage=Chroma.from_documents(split,OllamaEmbeddings()) #initializing our vector storage

KeyboardInterrupt: 